# Use sedona to read various geospatial data format

To work with geospatial data, it's essential to read and write data in geospatial data format. The **full list of the constructor for the geo data types** can be found [here](https://sedona.apache.org/1.6.1/api/sql/Constructor/)
For sedona version `1.6.1`, it supports at least 10 formats:

In this tutorial, we will show example of sedona to read various geospatial data format such as:
- plain text string without format
- csv/tsv
- wkt/wkb
- Ewkt/Ekb
- GML
- KML
- geojson
- shape file
- GeoHash
- geoparquet
- pbf
- ETC

This website: https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-referentiel-geographique/export/ shows how french ministry share their geospatial data.

## Geospatial data 

To be able to use sedona to do geospatial operations (e.g calculate distance, area hierarchy, etc.), we need to construct geo dataframe first. A geo dataframe contains one or more columns of below type:
- Point : a point on the map with a (x,y) coordinates
- Line: two point which can form a line
- Polygon: a list of point which can form a polygon

The `geometry column` must be able to express these data types.



We will also evaluate the performance(e.g. storage space, processing speed) of each format

In [1]:
from sedona.spark import *
import geopandas as gpd
from pyspark.sql.functions import trim, col
from pathlib import Path
import json

from ipyleaflet import Map, basemaps, basemap_to_tiles, MarkerCluster, Marker, AwesomeIcon
from ipywidgets import Layout
import numpy as np

In [2]:
# get the project root dir
project_root_dir = Path.cwd().parent.parent
data_dir = f"{project_root_dir}/data"

In [3]:
# build a sedona session (sedona = 1.6.1)
jar_folder = Path(f"{project_root_dir}/jars/sedona-35-213-161")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.6.1) offline
config = SedonaContext.builder() \
    .master("local[*]") \
    .config('spark.jars', jar_path). \
    getOrCreate()



In [4]:
# create a sedona context
sedona = SedonaContext.create(config)
sc = sedona.sparkContext


In [5]:
spark = sedona.getActiveSession()

In [6]:
# this sets the encoding of shape files
sc.setSystemProperty("sedona.global.charset", "utf8")

## 1. Read geospatial data

Sedona can read various geospatial data format
- CSV/TSV
- wkt/wkb (Ewkt/Ekb)
- geojson

In below example, we will read a normal csv file which contains two column x, y. You can notice the content of the csv is `plain text` string.

### 1.1 Point example

In below example, we will construct a geo dataframe which contains a **Point** column

In [7]:
point_file_path = f"{data_dir}/csv/test_points.csv"

# read a normal csv
raw_point_df = sedona.read.format("csv").\
          option("delimiter",",").\
          option("header","false").\
          load(point_file_path)

raw_point_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [8]:
raw_point_df.show(5)

+---+-----+
|_c0|  _c1|
+---+-----+
|1.1|101.1|
|2.1|102.1|
|3.1|103.1|
|4.1|104.1|
|5.1|105.1|
+---+-----+
only showing top 5 rows



In [9]:
# create a temp view
raw_point_df.createOrReplaceTempView("p_raw_table")

In [10]:
# we cast the string type to decimal first, then we use `ST_Point` function to build a geometry column by using the two column in the csv file 
geo_point_df = sedona.sql("select ST_Point(cast(p_raw_table._c0 as Decimal(24,20)), cast(p_raw_table._c1 as Decimal(24,20))) as point from p_raw_table")
geo_point_df.printSchema()

root
 |-- point: geometry (nullable = true)



In [11]:
geo_point_df.show(5,truncate=False)

+-----------------+
|point            |
+-----------------+
|POINT (1.1 101.1)|
|POINT (2.1 102.1)|
|POINT (3.1 103.1)|
|POINT (4.1 104.1)|
|POINT (5.1 105.1)|
+-----------------+
only showing top 5 rows



### 1.2 Line example

To create a line type, we can use the constructor **ST_LineStringFromText (Text:string, Delimiter:char)**. In below example, we can notice it takes a list of gps coordinates, then it returns a geometry column.

In [12]:
geo_line_df = sedona.sql("SELECT ST_LineStringFromText('-74.0428197,40.6867969,-74.0421975,40.6921336,-74.0508020,40.6912794', ',') AS line")
geo_line_df.printSchema()

root
 |-- line: geometry (nullable = true)



In [13]:
geo_line_df.show(5,truncate=False)

+----------------------------------------------------------------------------------+
|line                                                                              |
+----------------------------------------------------------------------------------+
|LINESTRING (-74.0428197 40.6867969, -74.0421975 40.6921336, -74.050802 40.6912794)|
+----------------------------------------------------------------------------------+



### 1.3 Polygon example

We have seen the below example for the section 1. We will use the constructor **ST_GeomFromText()**

In [23]:
us_county_file_path = f"{data_dir}/csv/county_small.tsv"

In [26]:
raw_df = sedona.read.format("csv").option("delimiter", "\t").option("header", "false").load(us_county_file_path)
raw_df.show()

+--------------------+---+---+--------+-----+-----------+--------------------+---+---+-----+----+-----+----+----+----------+--------+-----------+------------+
|                 _c0|_c1|_c2|     _c3|  _c4|        _c5|                 _c6|_c7|_c8|  _c9|_c10| _c11|_c12|_c13|      _c14|    _c15|       _c16|        _c17|
+--------------------+---+---+--------+-----+-----------+--------------------+---+---+-----+----+-----+----+----+----------+--------+-----------+------------+
|POLYGON ((-97.019...| 31|039|00835841|31039|     Cuming|       Cuming County| 06| H1|G4020|NULL| NULL|NULL|   A|1477895811|10447360|+41.9158651|-096.7885168|
|POLYGON ((-123.43...| 53|069|01513275|53069|  Wahkiakum|    Wahkiakum County| 06| H1|G4020|NULL| NULL|NULL|   A| 682138871|61658258|+46.2946377|-123.4244583|
|POLYGON ((-104.56...| 35|011|00933054|35011|    De Baca|      De Baca County| 06| H1|G4020|NULL| NULL|NULL|   A|6015539696|29159492|+34.3592729|-104.3686961|
|POLYGON ((-96.910...| 31|109|00835876|31109| 

In [30]:
raw_poly_df = raw_df.select("_c0","_c6").withColumnRenamed("_c0","county_polygon").withColumnRenamed("_c6","county_name")
raw_poly_df.createOrReplaceTempView("gon_raw_table")
raw_poly_df.show(5)

+--------------------+----------------+
|      county_polygon|     county_name|
+--------------------+----------------+
|POLYGON ((-97.019...|   Cuming County|
|POLYGON ((-123.43...|Wahkiakum County|
|POLYGON ((-104.56...|  De Baca County|
|POLYGON ((-96.910...|Lancaster County|
|POLYGON ((-98.273...| Nuckolls County|
+--------------------+----------------+
only showing top 5 rows



In [29]:
raw_poly_df.printSchema()

root
 |-- county_polygon: string (nullable = true)
 |-- county_name: string (nullable = true)



In [32]:
geo_polygon_df=sedona.sql("select ST_GeomFromText(gon_raw_table.county_polygon) as county_shape, gon_raw_table.county_name from gon_raw_table")
geo_polygon_df.show(5)

+--------------------+----------------+
|        county_shape|     county_name|
+--------------------+----------------+
|POLYGON ((-97.019...|   Cuming County|
|POLYGON ((-123.43...|Wahkiakum County|
|POLYGON ((-104.56...|  De Baca County|
|POLYGON ((-96.910...|Lancaster County|
|POLYGON ((-98.273...| Nuckolls County|
+--------------------+----------------+
only showing top 5 rows



In [33]:
geo_polygon_df.printSchema()

root
 |-- county_shape: geometry (nullable = true)
 |-- county_name: string (nullable = true)



## 1.4 Read wkt and wkb file

**Well-known text (WKT)** is a `text markup language` for representing vector geometry objects on a map and spatial reference systems of spatial objects. A binary equivalent, known as **well-known binary (WKB)** is used to transfer and store the same information for geometry objects.

Geometries in a `WKT and WKB` file always occupy a single column no matter how many coordinates they have. Sedona provides `WktReader and WkbReader` to create generic SpatialRDD. Then we need to convert the spatial rdd to dataframe.

> You must use the wkt reader to read wkt file, and wkb reader to read wkb file.

For `EWKT/EWKB`, we just have a extra column `SRID(Spatial Reference Identifier)  code` compare to WKT

```sql
SELECT ST_AsText(ST_GeomFromEWKT('SRID=4269;POINT(40.7128 -74.0060)'))

# output example
# POINT(40.7128 -74.006)
```

In [34]:
us_county_wkb_file_path = f"{data_dir}/csv/county_small_wkb.tsv"

In [35]:
from sedona.core.formatMapper import WkbReader
from sedona.utils.adapter import Adapter

In [36]:
# The WKT string starts from Column 0
wkbColumn = 0 
allowTopologyInvalidGeometries = True
skipSyntaxInvalidGeometries = False

spatialRdd = WkbReader.readToGeometryRDD(sedona.sparkContext, us_county_wkb_file_path, wkbColumn, allowTopologyInvalidGeometries, skipSyntaxInvalidGeometries)

In [40]:
geo_county_wkb_df = Adapter.toDf(spatialRdd,sedona).withColumnRenamed("geometry", "county_shape")
geo_county_wkb_df.show(5)

+--------------------+
|        county_shape|
+--------------------+
|POLYGON ((-97.019...|
|POLYGON ((-123.43...|
|POLYGON ((-104.56...|
|POLYGON ((-96.910...|
|POLYGON ((-98.273...|
+--------------------+
only showing top 5 rows



In [41]:
geo_county_wkb_df.printSchema()

root
 |-- county_shape: geometry (nullable = true)



### 1.5 Read geojson ((Geographic JavaScript Object Notation))

https://sedona.apache.org/1.6.1/tutorial/sql
Geojson has two different organization:
- single-line GeoJSON
- multi-line 

#### 1.5.1 Read single-line GeoJSON

In the single-line geoJSON organization, each line is a separate, self-contained GeoJSON object. Below is an example

```json
{"type":"Feature","geometry":{"type":"Point","coordinates":[102.0,0.5]},"properties":{"prop0":"value0"}}
{"type":"Feature","geometry":{"type":"LineString","coordinates":[[102.0,0.0],[103.0,1.0],[104.0,0.0],[105.0,1.0]]},"properties":{"prop0":"value1"}}
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[100.0,0.0],[101.0,0.0],[101.0,1.0],[100.0,1.0],[100.0,0.0]]]},"properties":{"prop0":"value2"}}
```
You can notice that each line starts with `{` ends with `}`, which means it's a self-contained json object.

> This format is efficient for processing large datasets, because each line is an independent GeoJSON Feature which can be processed in parallel.  

In [14]:
us_county_json_file_path = f"{data_dir}/geojson/us_county.json"

In [15]:
raw_json_df = sedona.read.format("geojson").load(us_county_json_file_path)
raw_json_df.show(5)

+--------------------+--------------------+-------+
|            geometry|          properties|   type|
+--------------------+--------------------+-------+
|POLYGON ((-87.621...|{1500000US0107701...|Feature|
|POLYGON ((-85.719...|{1500000US0104502...|Feature|
|POLYGON ((-86.000...|{1500000US0105500...|Feature|
|POLYGON ((-86.574...|{1500000US0108900...|Feature|
|POLYGON ((-85.382...|{1500000US0106904...|Feature|
+--------------------+--------------------+-------+
only showing top 5 rows



In [16]:
raw_json_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- AFFGEOID: string (nullable = true)
 |    |-- ALAND: long (nullable = true)
 |    |-- AWATER: long (nullable = true)
 |    |-- BLKGRPCE: string (nullable = true)
 |    |-- COUNTYFP: string (nullable = true)
 |    |-- GEOID: string (nullable = true)
 |    |-- LSAD: string (nullable = true)
 |    |-- NAME: string (nullable = true)
 |    |-- STATEFP: string (nullable = true)
 |    |-- TRACTCE: string (nullable = true)
 |-- type: string (nullable = true)



### 1.5.2 Read multi-line GeoJSON

The multi-line GeoJSON use a global `{ "type": "FeatureCollection", }` to encapsulate all geo features in one JSON object. Below is an example

```json
{ "type": "FeatureCollection",
    "features": [
      { "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [102.0, 0.5]},
        "properties": {"prop0": "value0"}
        },
      { "type": "Feature",
        "geometry": {
          "type": "LineString",
          "coordinates": [
            [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
            ]
          },
        "properties": {
          "prop0": "value1",
          "prop1": 0.0
          }
        },
      { "type": "Feature",
         "geometry": {
           "type": "Polygon",
           "coordinates": [
             [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0],
               [100.0, 1.0], [100.0, 0.0] ]
             ]
         },
         "properties": {
           "prop0": "value2",
           "prop1": {"this": "that"}
           }
         }
       ]
}
```

Multiline format is preferable for scenarios where files need to be human-readable or manually edited.

As the entire file is considered as a single json object, it's hard to process in parallel

In [24]:
from pyspark.sql.functions import expr

multi_line_json_file_path = f"{data_dir}/geojson/multi_lines.json"

df_raw = sedona.read.format("geojson").option("multiLine", "true").load(multi_line_json_file_path)
          
df_raw.show(5,truncate=False)
df_raw.printSchema()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|features                                                                                                                                                                                            |type             |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|[{POINT (102 0.5), {value0, NULL}, Feature}, {LINESTRING (102 0, 103 1, 104 0, 105 1), {value1, 0.0}, Feature}, {POLYGON ((100 0, 101 0, 101 1, 100 1, 100 0)), {value2, {"this":"that"}}, Feature}]|FeatureCollection|
+-----------------------------------------------------------------------------------------------------------------------------------

As the entire file is a single json object(FeatureCollection), all features are consider as items in one big array, To get one feature per row, we need to explode the array envelope.

In [23]:
df = df_raw.selectExpr("explode(features) as features").select("features.*")
 

df.show()
df.printSchema()

+--------------------+--------------------+-------+
|            geometry|          properties|   type|
+--------------------+--------------------+-------+
|     POINT (102 0.5)|      {value0, NULL}|Feature|
|LINESTRING (102 0...|       {value1, 0.0}|Feature|
|POLYGON ((100 0, ...|{value2, {"this":...|Feature|
+--------------------+--------------------+-------+

root
 |-- geometry: geometry (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- prop0: string (nullable = true)
 |    |-- prop1: string (nullable = true)
 |-- type: string (nullable = true)



## 1.6 Read GML

**GML(Geography Markup Language)** is an `XML based encoding standard` for geographic information developed by the `OpenGIS Consortium (OGC)`. You can find the official doc [here](https://www.ogc.org/publications/standard/gml/)
It has three major version:
- GML 1
- GML 2
- GML 3

Sedona(<v1.6.1) only supports `GML1 and GML2` for now.  

In [33]:
gml_sample = """
<gml:LineString srsName="EPSG:4269">
        <gml:coordinates>
            -71.16028,42.258729
            -71.160837,42.259112
            -71.161143,42.25932
        </gml:coordinates>
</gml:LineString>
"""

gml_df = sedona.sql(f"SELECT ST_GeomFromGML('{gml_sample}') as line")

In [35]:
gml_df.show(5, truncate=False)
gml_df.printSchema()

+---------------------------------------------------------------------------+
|line                                                                       |
+---------------------------------------------------------------------------+
|LINESTRING (-71.16028 42.258729, -71.160837 42.259112, -71.161143 42.25932)|
+---------------------------------------------------------------------------+

root
 |-- line: geometry (nullable = true)



## 1.7 Read KML

**Keyhole Markup Language (KML)** is an `XML notation` for expressing geographic annotation and visualization within two-dimensional maps and three-dimensional Earth browsers. `KML was developed for use with Google Earth`, which was originally named `Keyhole Earth Viewer`.

A complete kml file example.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
<Placemark>
  <name>New York City</name>
  <description>New York City</description>
  <Point>
    <coordinates>-74.006393,40.714172,0</coordinates>
  </Point>
</Placemark>
</Document>
</kml>
``` 

You can notice the coordinates has three attributes (longitude,latitude,altitude), with the altitude coordinates, we can build a 3D map.




In [40]:
kml_sample = """
<Point>
    <coordinates>-74.006393,40.714172,0</coordinates>
</Point>
"""

kml_df = sedona.sql(f"SELECT ST_GeomFromKML('{kml_sample}') as point")

In [41]:
kml_df.show(5, truncate=False)
kml_df.printSchema()

+----------------------------+
|point                       |
+----------------------------+
|POINT (-74.006393 40.714172)|
+----------------------------+

root
 |-- point: geometry (nullable = true)



## 1.8 Read Geohash

A **Geohash is a unique identifier** of a specific region on the Earth. The basic idea is that the Earth is divided into regions of user-defined size and each region is assigned a unique id, which is called its Geohash. 

You can try to get the geohash of any region with this web site: https://www.movable-type.co.uk/scripts/geohash.html

In [42]:
geohash_sample = "s00twy01mt"


geohash_df = sedona.sql(f"SELECT ST_GeomFromGeoHash('{geohash_sample}') as polygon")

In [43]:
geohash_df.show(5, truncate=False)
geohash_df.printSchema()

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|polygon                                                                                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|POLYGON ((0.9999918937683105 0.9999972581863403, 0.9999918937683105 1.0000026226043701, 1.0000026226043701 1.0000026226043701, 1.0000026226043701 0.9999972581863403, 0.9999918937683105 0.9999972581863403))|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 1.9 Read shape file

A shapefile is a `vector data file format` commonly used for geospatial analysis. Shapefiles store the location, geometry, and attribution of `point, line, and polygon` features.

You can visit the wiki page via this [link](https://en.wikipedia.org/wiki/Shapefile).


In [50]:
airports_shape_file_path = f"{data_dir}/shapefile/airports_shape"

# read communes shape file
airports_rdd = ShapefileReader.readToGeometryRDD(sc, airports_shape_file_path)
airports_df = Adapter.toDf(airports_rdd, sedona)


In [51]:
airports_df.printSchema()
airports_df.show(1, truncate=False)

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: string (nullable = true)

+---------------------------------------------+---------+--------------------------------------------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------------------------------------------+------------------------------------------------------------------------------------------------------------

In [53]:
airports_df.select("geometry","type","name","location","iata_code","gps_code").show(5, truncate=False)

+---------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|geometry                                     |type                                              |name                                     

## 1.10 Read GeoParquet

GeoParquet is an **Open Geospatial Consortium (OGC) standard** that adds interoperable geospatial types `(Point, Line, Polygon)` to Parquet. Currently(27/11/2024), the stable version is 1.1.0.

You can find the official site of geo-parquet [here](https://geoparquet.org/).

You can find the current version data format specification [here](https://geoparquet.org/releases/v1.1.0/). The supported geo type are:
- point
- line
- polygon
- multipoint
- multiline
- multipolygon

In [55]:
airports_parquet_path = f"{data_dir}/parquet/airports"

airports_parquet_df= sedona.read.format("geoparquet").load(airports_parquet_path)

In [57]:
airports_parquet_df.printSchema()
airports_parquet_df.show(5)

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: string (nullable = true)

+--------------------+---------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+---------+
|            geometry|scalerank|          featurecla|                type|                name|abbrev|            location|            gps_code|           iata_code|           wikipedia|natlscale|
+--------------------+---------+--------------------+--------------------+--------------------+------+--------------------+--------------------+------------------

### 1.10.1 Read metadata of geo parquet



In [59]:
metadata_df = sedona.read.format("geoparquet.metadata").load(airports_parquet_path)
metadata_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- version: string (nullable = true)
 |-- primary_column: string (nullable = true)
 |-- columns: map (nullable = true)
 |    |-- key: string
 |    |-- value: struct (valueContainsNull = true)
 |    |    |-- encoding: string (nullable = true)
 |    |    |-- geometry_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- bbox: array (nullable = true)
 |    |    |    |-- element: double (containsNull = true)
 |    |    |-- crs: string (nullable = true)
 |    |    |-- covering: string (nullable = true)



In [61]:
metadata_df.show(truncate=False, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------
 path           | file:/C:/Users/PLIU/Documents/git/GeoParquetAndSedona/data/parquet/airports/part-00000-b950f0ae-c815-4a20-b44d-5556dcd46339-c000.snappy.parquet 
 version        | 1.1.0                                                                                                                                           
 primary_column | geometry                                                                                                                                        
 columns        | {geometry -> {WKB, [Point], [-175.135635, -53.005069825517666, 178.5600483699593, 71.289299], null, NULL}}                                      



## 1.11 Read data from JDBC datasource

As spark offers jdbc reader, so sedona can also access various database source via JDBC. There is no major difference between read a file and read a database table.

## 1.12 Read data from geopackage

A **GeoPackage** is an SQLite Database file a **specific database schema**, the filename extension is `.gpkg`. If you are unsure whether a file is an SQLite database, you can use a binary or text editor to view the starting bytes of the file and see if they state `SQLite format 3`.

In the next major release of sedona(1.7), the `GeoPackage` format will be supported.


## 2. Write geospatial data via sedona

Natively, sedona allows user to write geospatial data in GeoJson, wkt/wkb in csv, geoparquet

 

In [62]:
airports_parquet_path = f"{data_dir}/parquet/airports"

airports_parquet_df= sedona.read.format("geoparquet").load(airports_parquet_path)

In [63]:
airports_parquet_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: string (nullable = true)



In [64]:
output_df = airports_parquet_df.select("geometry","type","name","location","iata_code","gps_code")


In [65]:
output_df.show()
output_df.printSchema()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            geometry|                type|                name|            location|           iata_code|            gps_code|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|POINT (113.935016...|major            ...|Hong Kong Int'l  ...|terminal         ...|HKG              ...|VHHH             ...|
|POINT (121.231370...|major            ...|Taoyuan          ...|terminal         ...|TPE              ...|RCTP             ...|
|POINT (4.76437693...|major            ...|Schiphol         ...|terminal         ...|AMS              ...|EHAM             ...|
|POINT (103.986413...|major            ...|Singapore Changi ...|terminal         ...|SIN              ...|WSSS             ...|
|POINT (-0.4531566...|major            ...|London Heathrow  ...|parking          ...|LHR              ..

In [66]:
print(f"Total row number: {output_df.count()}")

Total row number: 281


### 2.1 Write as GeoJSON

Since `v1.6.1`, the GeoJSON data source in Sedona can be used to save a Spatial DataFrame to a `single-line JSON` file, with geometries written in GeoJSON format.

In [74]:
geo_json_out_path= f"{data_dir}/tmp/geojson_sample"
output_df.write.format("geojson").save(geo_json_out_path)

### 2.2 Write as wkt 

In [82]:
wkt_out_df = output_df.selectExpr("ST_AsText(geometry) as geo_string","type", "name", "location", "iata_code", "gps_code")
wkt_out_df.printSchema()
wkt_out_df.show(5)

root
 |-- geo_string: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- gps_code: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          geo_string|                type|                name|            location|           iata_code|            gps_code|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|POINT (113.935016...|major            ...|Hong Kong Int'l  ...|terminal         ...|HKG              ...|VHHH             ...|
|POINT (121.231370...|major            ...|Taoyuan          ...|terminal         ...|TPE              ...|RCTP             ...|
|POINT (4.76437693...|major            ...|Schiphol         ...|terminal         ...|AMS              ...|EHAM         

In [83]:
wkt_sample_path = f"{data_dir}/tmp/wkt_sample"
wkt_out_df.write.csv(wkt_sample_path)

## 2.3 Write as wkb

As wkb is binary, we can't write in csv. But we can write in parquet native (which is not geoparquet) 

In [78]:
wkb_out_df = output_df.selectExpr("ST_AsBinary(geometry) as geo_bin","type", "name", "location", "iata_code", "gps_code")
wkb_out_df.printSchema()
wkb_out_df.show(5)

root
 |-- geo_bin: binary (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- gps_code: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             geo_bin|                type|                name|            location|           iata_code|            gps_code|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[01 01 00 00 00 5...|major            ...|Hong Kong Int'l  ...|terminal         ...|HKG              ...|VHHH             ...|
|[01 01 00 00 00 2...|major            ...|Taoyuan          ...|terminal         ...|TPE              ...|RCTP             ...|
|[01 01 00 00 00 0...|major            ...|Schiphol         ...|terminal         ...|AMS              ...|EHAM            

In [80]:
wkb_sample_path = f"{data_dir}/tmp/wkb_sample"
wkb_out_df.write.parquet(wkb_sample_path)


### 2.4 Write as geoparquet


In [81]:
geoparquet_sample_path = f"{data_dir}/tmp/geoparquet_sample"
output_df.write.format("geoparquet").option("geoparquet.version","1.1.0").save(geoparquet_sample_path)

#### 2.4.1 Custom metadata in geo parquet

As parquet allows us to insert custom metadata, so geo parquet has this ability too.

The most used metadata in geo parquet is:
  - crs coordinates for geometry column
  - covering metadata: For each geometry column, a top-level struct column containing xmin, ymin, xmax,ymax. The `covering` field specifies a bounding box column to help accelerate spatial data retrieval 


##### Integrated CRS coordinates for geometry columns

The maps of the surface of the earth is in two dimensions. But, as you know, the world is actually a three-dimensional globe. So we have to use a method called a `map projection` to render it as a flat surface. We use a coordinate reference system (CRS) to show how the projected points correspond to real locations on Earth.

For example, the coordinate (Latitude: 48.8566, Longitude: 2.3522) is in Paris in the `WGS84(ESPG:4326)` CRS. The same point in `EPSG:3857` (e.g. OSM) would be (X: 261845.83,Y: 6250561.88)

In [84]:
output_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- gps_code: string (nullable = true)



In [87]:
crs_code_for_geometry ="""
{
    "$schema": "https://proj.org/schemas/v0.7/projjson.schema.json",
    "type": "GeographicCRS",
    "name": "WGS 84",
    "datum_ensemble": {
        "name": "World Geodetic System 1984 ensemble",
        "members": [
            {
                "name": "World Geodetic System 1984 (Transit)",
                "id": {
                    "authority": "EPSG",
                    "code": 1166
                }
            },
            {
                "name": "World Geodetic System 1984 (G730)",
                "id": {
                    "authority": "EPSG",
                    "code": 1152
                }
            },
            {
                "name": "World Geodetic System 1984 (G873)",
                "id": {
                    "authority": "EPSG",
                    "code": 1153
                }
            },
            {
                "name": "World Geodetic System 1984 (G1150)",
                "id": {
                    "authority": "EPSG",
                    "code": 1154
                }
            },
            {
                "name": "World Geodetic System 1984 (G1674)",
                "id": {
                    "authority": "EPSG",
                    "code": 1155
                }
            },
            {
                "name": "World Geodetic System 1984 (G1762)",
                "id": {
                    "authority": "EPSG",
                    "code": 1156
                }
            },
            {
                "name": "World Geodetic System 1984 (G2139)",
                "id": {
                    "authority": "EPSG",
                    "code": 1309
                }
            }
        ],
        "ellipsoid": {
            "name": "WGS 84",
            "semi_major_axis": 6378137,
            "inverse_flattening": 298.257223563
        },
        "accuracy": "2.0",
        "id": {
            "authority": "EPSG",
            "code": 6326
        }
    },
    "coordinate_system": {
        "subtype": "ellipsoidal",
        "axis": [
            {
                "name": "Geodetic latitude",
                "abbreviation": "Lat",
                "direction": "north",
                "unit": "degree"
            },
            {
                "name": "Geodetic longitude",
                "abbreviation": "Lon",
                "direction": "east",
                "unit": "degree"
            }
        ]
    },
    "scope": "Horizontal component of 3D system.",
    "area": "World.",
    "bbox": {
        "south_latitude": -90,
        "west_longitude": -180,
        "north_latitude": 90,
        "east_longitude": 180
    },
    "id": {
        "authority": "EPSG",
        "code": 4326
    }
}"""

In [95]:
geoparquet_crs_sample_path = f"{data_dir}/tmp/geoparquet_crs_sample"
output_df.write.mode("overwrite").format("geoparquet").option("geoparquet.crs.geometry",crs_code_for_geometry).option("geoparquet.version","1.1.0").save(geoparquet_crs_sample_path)

In [97]:
geo_crs_meta = sedona.read.format("geoparquet.metadata").load(geoparquet_crs_sample_path)

geo_crs_meta.show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Covering metadata

Since `v1.6.1`, Sedona supports writing the **covering field** to geometry column metadata. The covering field specifies a bounding box column to help accelerate spatial data retrieval. The bounding box column should be a top-level struct column containing `xmin, ymin, xmax, ymax` columns. If the DataFrame you are writing contains such columns, you can specify 
`.option("geoparquet.covering.<geometryColumnName>", "<coveringColumnName>")` option to write covering metadata to GeoParquet files:

In [91]:
df_with_bbox = output_df.withColumn("bbox",  expr("struct(ST_XMin(geometry) AS xmin, ST_YMin(geometry) AS ymin, ST_XMax(geometry) AS xmax, ST_YMax(geometry) AS ymax)"))
df_with_bbox.show(5, truncate=False)

+---------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|geometry                                     |type       

In [93]:
geoparquet_bbox_sample_path = f"{data_dir}/tmp/geoparquet_bbox_sample"

df_with_bbox.write.format("geoparquet").option("geoparquet.covering.geometry", "bbox").save(geoparquet_bbox_sample_path)

In [94]:
geo_crs_meta = sedona.read.format("geoparquet.metadata").load(geoparquet_bbox_sample_path)

geo_crs_meta.show(truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 path           | file:/C:/Users/PLIU/Documents/git/GeoParquetAndSedona/data/tmp/geoparquet_bbox_sample/part-00000-ff075f90-66d4-40ee-bb5c-5906ee6c9042-c000.snappy.parquet                                                    
 version        | 1.1.0                                                                                                                                                                                                        
 primary_column | geometry                                                                                                                                                                                                     
 columns        | {geometry -> {WKB, [Point], [-175.135635, -53.005069825517666, 178.5600483699593, 71.2

### 2.5 Write to 

Unfortunately, the Spark SQL JDBC data source doesn't support creating geometry types in PostGIS using the 'createTableColumnTypes' option. Only the Spark built-in types are recognized. This means that you'll need to manage your PostGIS schema separately from Spark. One way to do this is to create the table with the correct geometry column before writing data to it with Spark. Alternatively, you can write your data to the table using Spark and then manually alter the column to be a geometry type afterward.

Postgis uses EWKB to serialize geometries. If you convert your geometries to EWKB format in Sedona you don't have to do any additional conversion in Postgis.

```python
# create a table in my_postgis_db first. create table my_table (id int8, geom geometry);

# Use jdbc to insert data, but don't let Spark recreate the table.
# you can use the option("truncate","true")
df.withColumn("geom", expr("ST_AsEWKB(geom)")
    .write.format("jdbc")
    .option("truncate","true")
    .save()

# If you didn't create the table before writing you can change the type afterward in your my_postgis_db
# alter table my_table alter column geom type geometry;

```

### 2.6 Write to other formats

Sedona does not support to write in other formats natively. But there exist many tools which can convert geojson or wkt to other formats such as shapefile, GML, etc.

## 3. Compare different data format disk usage 

Compare the result of shape file and geo parquet, we don't gain too many things

| file format    | disk space | duration (in sec) |
|----------------|------------|-------------------|
| wkt_in_csv     | 23.7Kb     | 1,45              |
| wkb_in_parquet | 24.1Kb     | 1,32              |
| geojson        | 272Kb      | 1,60              |
| geoparquet     | 24.5Kb     | 1.21              |      

# Appendix

## Coordinate Reference System (CRS)

The maps of the surface of the earth is in two dimensions. But, as you know, the world is actually a three-dimensional globe. So we have to use a method called a `map projection` to render it as a flat surface. We use a coordinate reference system (CRS) to show how the projected points correspond to real locations on Earth.

A Coordinate Reference System (CRS) is fundamental in geospatial data and GIS (Geographic Information Systems) because it defines how spatial data is represented on the Earth's surface. Without a proper CRS, geographic data from different sources would not align correctly, leading to errors and confusion in analysis, visualization, and decision-making.

## **WGC**(World Geodetic reference System)

It's used globally for mapping and navigation. The most widely known version is **WGS84**, which is the standard used by GPS. It defines the shape of the Earth (ellipsoid), the datum, and the coordinate system.

Key Features:

  - Datum: WGS84 Datum, a global standard for latitude, longitude, and ellipsoidal height.
  - Coordinate System: A geographic coordinate system (latitude/longitude) on the WGS84 ellipsoid.
   - Usage: Primarily used for GPS and satellite navigation systems.
      - 
 > WGS84 is often represented as EPSG:4326 in the EPSG registry, a geographic CRS with latitude and longitude coordinates.
 

## OGC(Open Geospatial Consortium)

**OGC** is an international organization that sets standards for geospatial and location-based services. It doesn’t define specific CRS codes but provides frameworks and specifications for managing and using CRS in geospatial systems.

The OGC CRS URNs are part of its standard for referencing coordinate systems.

```text
urn:ogc:def:crs:EPSG::4326
```

## EPSG(European Petroleum Survey Group)

**EPSG** is a registry maintained by the `International Association of Oil & Gas Producers (IOGP)`. It provides a database of `standardized codes for various CRS definitions`, including geographic, projected, and vertical coordinate systems.

EPSG codes are numeric identifiers for specific CRS or transformation. For example:
 - **EPSG:4326**: `WGS84` in geographic coordinates (latitude/longitude).
 - **EPSG:3857**: `WGS84` Web Mercator projection (used by most web mapping services).

> Widely used in GIS software (e.g., QGIS, ArcGIS) and geospatial APIs.


